---
Vamos implementar um sistema de recomendação utilizando Cosine-similarity. Vamos utilizar o dataset criado no collab [anterior](https://colab.research.google.com/drive/1W0R5oW4K5goml3EOddxYjotUZegLJF-V?usp=sharing). Entretando precisamos trabalhar com os dados, para facilitar a utilização deste metodo. Faremos isto neste collab.

---



### Preparação de data


Temos os dados em csv de séries e filmes na Netflix em um unico dataset, criado no collab anterior. Para fazer um sistema de recomendação precisamos unir todo nosso conteúdo que achamos relevante em apenas uma string. Também precisamos eliminar stop words( palavras que não acrescentam ao texto como E, pois, A). Vamos retirar pontuações também para não pesar na nossa matriz. Por fim, vamos eliminar as colunas redundantes, deixando só o Título e o texto concatenado.



O dataset utilizado é o do collab de análise e para facilitar a importação, aloquei o mesmo no meu gitbub. Existem várias maneiras de agregar este dados, utilizei isto por praticidade.

Importando Bibliotecas utilizadas


In [3]:
import pandas as pd
import calendar
from datetime import datetime
import calendar

Carregando os dados.

In [7]:
folder = "https://raw.githubusercontent.com/arthurtorres/Projetos/master/Netflix/data_informacao/dataNetflixLimpa.csv"

dataNetflix = pd.read_csv(folder)


Como ID não é uma caractéristica relevante,podemos remove-lo do dataset.

In [8]:
dataInicio = dataNetflix.drop(columns =["Id"])
dataInicio.head(5)

,Tipo,Titulo,Diretor,Elenco,Pais,Ano_Lancamento,Classificação_Indicativa,Duração,Genero,Sinopse,Ano_Netflix,Mês_Netflix,Dia_Netflix
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,2019,9,9
1,Movie,Jandino: Whatever it Takes,Diretor Desconhecido,Jandino Asporaat,United Kingdom,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,2016,9,9
2,TV Show,Transformers Prime,Diretor Desconhecido,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",2018,9,8
3,TV Show,Transformers: Robots in Disguise,Diretor Desconhecido,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,2018,9,8
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,2017,9,8


Acredito que o mês e o dia da semana  que a serie foi adicionada pode ser um fator para alguns, logo adicionarei esta coluna, utilizando para isto a biblioteca calendar.

In [9]:
dataInicio['Month_Netflix'] = dataInicio['Mês_Netflix'].apply(lambda x: calendar.month_abbr[x])

dataInicio['Weekday'] = pd.to_datetime(dict(year =dataInicio['Ano_Netflix'], month = dataInicio['Mês_Netflix'],
                     day=dataInicio['Dia_Netflix'])).apply(lambda x: calendar.day_name[x.weekday()])


Vamos aproveitar esta transformação e ver a quantidade de filmes lançadas por dia da semana.

In [10]:
dataInicio.Weekday.value_counts()

Friday       1910
Thursday      854
Tuesday       845
Wednesday     758
Monday        705
Saturday      598
Sunday        564
Name: Weekday, dtype: int64

Como era de se esperar, sexta feira tem praticamente um terço dos dados.

Vamos usar uma função para remover espaçõs sobresalentes de alguams coluans de interesse

In [11]:
def limpezaVirgulaDataset(dataset,coluna) :
  """
  Função que pega uma coluna do dataset e retira espaços em branco pós e anterior a virgulas
  """
  dataset[coluna]=dataset[coluna].str.replace(", ",",").str.replace(" ,", ",").str.replace(" ","") 




In [12]:
colunaInt = ["Diretor","Elenco","Pais","Genero","Duração"]

for coluna in colunaInt :
  limpezaVirgulaDataset(dataInicio,coluna)



Por fim, precisamos apenas juntar os dados em uma coluna, com a concatenização dos dados que achamos ter impacto. Pode analisar que o Mês que a serie foi adicionada esta duplicado, pois quero dar mais valor a esta classificação.

In [13]:
def JunçãoData(data) :
  data["objetivo"] = data["Tipo"]+data["Elenco"]+data["Diretor"]+data["Pais"]+data["Genero"]+data["Duração"]+data["Genero"] +data["Month_Netflix"]*2 +data["Classificação_Indicativa"]+data["Weekday"]



In [14]:
JunçãoData(dataInicio)

Por fim, falta apenas reduzir o dataset para apenas as colunas de interesse Titulo e Objetivo)

In [15]:
dataFinal = dataInicio[["Titulo","objetivo"]]



Com o dataset finalizado, podemos então exportar a data como queremos. Eu exportarei no github, numa pasta chamada data_informação, para poder utilizar no sistema de recomendação.

In [16]:
dataFinal.to_csv("data.csv")